In [1]:
from lesson_functions import *
import glob
from os.path import join
from sklearn.metrics import precision_score, classification_report, accuracy_score
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
import time
import pickle

# Functions

In [2]:
def non_shuffling_train_test_split(X, y, test_size=0.2):
    i = int((1 - test_size) * X.shape[0]) + 1
    X_train, X_test = np.split(X, [i])
    y_train, y_test = np.split(y, [i])
    return X_train, X_test, y_train, y_test

# Setup

## Parameters

In [3]:
data_dir = "./images/"

In [4]:
color_space = 'HLS'  # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 16  # HOG pixels per cell
cell_per_block = 2  # HOG cells per block
hog_channel = "ALL"  # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16)  # Spatial binning dimensions
hist_bins = 16  # Number of histogram bins
spatial_feat = True  # Spatial features on or off
hist_feat = True  # Histogram features on or off
hog_feat = True  # HOG features on or off

# Create feature vectors

## Get the paths of the training images

In [5]:
cars = glob.glob(join(data_dir, "vehicles/*/*.png"))
notcars = glob.glob(join(data_dir, "non-vehicles/*/*.png"))

## Create feature vectors

In [6]:
car_features = extract_features(cars, color_space=color_space,
                                spatial_size=spatial_size, hist_bins=hist_bins,
                                orient=orient, pix_per_cell=pix_per_cell,
                                cell_per_block=cell_per_block,
                                hog_channel=hog_channel, spatial_feat=spatial_feat,
                                hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(notcars, color_space=color_space,
                                   spatial_size=spatial_size, hist_bins=hist_bins,
                                   orient=orient, pix_per_cell=pix_per_cell,
                                   cell_per_block=cell_per_block,
                                   hog_channel=hog_channel, spatial_feat=spatial_feat,
                                   hist_feat=hist_feat, hog_feat=hog_feat)

/Users/seth.bunke/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


## Split and scale feature vector

In [7]:
car_X_train, car_X_test, car_y_train, car_y_test = \
    non_shuffling_train_test_split(np.array(car_features), np.ones(len(car_features)), test_size=0.2)

noncar_X_train, noncar_X_test, noncar_y_train, noncar_y_test = \
    non_shuffling_train_test_split(np.array(notcar_features), np.zeros(len(notcar_features)), test_size=0.2)

X_train = np.vstack((car_X_train, noncar_X_train))
y_train = np.hstack((car_y_train, noncar_y_train))
X_test = np.vstack((car_X_test, noncar_X_test))
y_test = np.hstack((car_y_test, noncar_y_test))

X_scaler = StandardScaler().fit(np.vstack((X_train, X_test)))
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

# Run fitting

## Testing

In [8]:
print('Using:', orient, 'orientations', pix_per_cell,
      'pixels per cell and', cell_per_block, 'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC
svc = LinearSVC(dual=True, C=0.01)
# Check the training time for the SVC
t = time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2 - t, 2), 'Seconds to train SVC...')
t = time.time()
p = svc.predict(X_test)
t2 = time.time()
# Check the prediction time
print(round(t2 - t, 4), 'Seconds to predict with SVC...')
# Check the score of the SVC
print('Test accuracy: {:.4f}'.format(accuracy_score(y_test, p)))
print("Test precision: {:.4f}".format(precision_score(y_test, p)))
print("")
print(classification_report(y_test, p, digits=4))

Using: 9 orientations 16 pixels per cell and 2 cells per block
Feature vector length: 1788
3.29 Seconds to train SVC...
0.0203 Seconds to predict with SVC...
Test accuracy: 0.9772
Test precision: 0.9635

             precision    recall  f1-score   support

        0.0     0.9914    0.9632    0.9771      1793
        1.0     0.9635    0.9915    0.9773      1758

avg / total     0.9776    0.9772    0.9772      3551



## Train final classifier with all data

In [9]:
X = np.vstack((X_train, X_test))
y = np.hstack((y_train, y_test))

t = time.time()
svc.fit(X, y)
t2 = time.time()
print(round(t2 - t, 2), 'Seconds to train final SVC...')

4.7 Seconds to train final SVC...


## Pickle the classifier and scaler

In [10]:
with open("svc.p", "wb") as ofile:
    pickle.dump([svc, X_scaler], ofile)